In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

In [2]:
from MyPooling import LogSumExpPool

In [9]:
pretrained = models.resnet50(pretrained=True)

class MyResNet(nn.Module):
    
    def __init__(self, my_pretrained_model):
        super(MyResNet, self).__init__()
        
        self.pretrained = my_pretrained_model
        self.pretrained.fc = nn.Identity()
        self.pretrained.avgpool = nn.Identity()
        self.transition = nn.Conv2d(2048, 2048, kernel_size=(2, 2), padding=16, bias=False)
        self.pooling = LogSumExpPool(r=5)
        self.prediction = nn.Linear(in_features=2048, out_features=8, bias=True)
    
    def forward(self, x):
        x = self.pretrained(x)
        x = self.transition(x[:,:,None,None])
        x = self.pooling(x)
        x = self.prediction(torch.flatten(x, start_dim=1))
        return x

model = MyResNet(my_pretrained_model=pretrained)

In [10]:
##### Вывод модели: восемь логитов
# model.eval()
# test_input = torch.rand(16, 3, 32, 32)
# model(test_input)